In [1]:
#r "nuget:HtmlAgilityPack, 1.11.65"
#r "nuget:ReverseMarkdown, 4.6.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.17.2"

using HtmlAgilityPack;
using ReverseMarkdown;
using Microsoft.Extensions.Configuration;
using System.IO;


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

public static var config = new ConfigurationBuilder()
          .AddJsonFile(Path.GetFullPath("secrets.json"), optional: false, reloadOnChange: true)
          .Build();

public static string OpenAIKey =  config["openai-key"];

Installed Packages HtmlAgilityPack, 1.11.65 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.FileExtensions, 8.0.1 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.17.2 ReverseMarkdown, 4.6.0

In [ ]:
var url = "https://www.docker.com/blog/wasm-vs-docker/";
var uri = new Uri(url);

// Get the URL specified
var webGet = new HtmlWeb();
var document = webGet.Load(uri);
var body =  document.DocumentNode.SelectSingleNode("/html/body");
var metaTags = document.DocumentNode.SelectNodes("//meta");
var hasMeta = metaTags != null;
var metaText = string.Empty;

body.OuterHtml

In [ ]:
body.OuterHtml

In [56]:
if(hasMeta)
{
   var sb = new StringBuilder();
   foreach(var item in metaTags)
   {
    sb.AppendLine(item.GetAttributeValue("property", "") + "|"+ item.GetAttributeValue("content", ""));
   }
   metaText = sb.ToString();
}

In [57]:
metaText

|
|IE=edge
|Learn what WebAssembly is, why to use it, and how Docker and Wasm work together.
|follow, index, max-snippet:-1, max-video-preview:-1, max-image-preview:large
og:locale|en_US
og:type|article
og:title|Wasm vs. Docker | Docker
og:description|Learn what WebAssembly is, why to use it, and how Docker and Wasm work together.
og:url|https://www.docker.com/blog/wasm-vs-docker/
og:site_name|Docker
article:publisher|https://www.facebook.com/docker.run
article:tag|containers
article:tag|Docker
article:tag|Docker Desktop
article:tag|vs
article:tag|WebAssembly (Wasm)
article:section|Products
og:updated_time|2024-08-05T09:23:49-07:00
og:image|https://www.docker.com/wp-content/uploads/2024/05/1300x1300_wasm-vs-docker-1024x1024.png
og:image:secure_url|https://www.docker.com/wp-content/uploads/2024/05/1300x1300_wasm-vs-docker-1024x1024.png
og:image:width|1024
og:image:height|1024
og:image:alt|Square illustration with white text on blue background saying Wasm vs. Docker
og:image:type|image/p

In [26]:
var config = new ReverseMarkdown.Config{
    UnknownTags = Config.UnknownTagsOption.Drop
};

var converter = new ReverseMarkdown.Converter(config);
string html = body.OuterHtml;

string result = converter.Convert(html);
result

# Wasm vs. Docker: Performant, Secure, and Versatile Containers

![](https://www.docker.com/wp-content/uploads/2024/03/Savannah-Ostrowski.webp)

[Savannah Ostrowski](https://www.docker.com/author/savannah-ostrowski/ "Posts by Savannah Ostrowski")

![](https://www.docker.com/wp-content/uploads/2024/04/Sohan-Profile-scaled.jpeg)

[Sohan Maheshwar](https://www.docker.com/author/sohan-mahehwar/ "Posts by Sohan Maheshwar")

Docker and WebAssembly (Wasm) represent two pivotal technologies that have reshaped the software development landscape. You’ve probably started to hear more about Wasm in the past few years as it has gained in popularity, and perhaps you’ve also heard about the benefits of using it in your application stack. This may have led you to think about the differences between Wasm and Docker, especially because the technologies work together so closely.

In this article, we’ll explore how these two technologies can work together to enable you to deliver consistent, efficient, an

In [60]:
static string ModelId = "gpt-3.5-turbo";

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder()
.AddOpenAIChatCompletion(ModelId, OpenAIKey);

Kernel kernel = builder.Build();

var textToSummarize = hasMeta ? metaText : result;


// Create and print out the prompt
string promptBody = $"""
    Consider a JSON schema for Article Summary that includes the following  properties: Author:string, Date:datetime, Title:string, Summary:string 
    
    Please summarize the the following text in 30 words or less for software engineers as the audience and output in json:
    {textToSummarize}

    # How to respond to this prompt
        - No other text, just the JSON data
    """;

// Create and print out the prompt
string promptMeta = $@"""
    Consider a JSON schema for Article Summary that includes the following  properties: Author:string, Date:datetime, Title:string, Summary:string 

    A key value par is represented as 
    key|value

    using the map below put the value of the corresponding key in the json document

    map
    og:description = Summary
    og:title = Title
    
    Extract the Article Summary from the key value pairs below and output in json:
    {textToSummarize}

    """;

string promp = hasMeta ? promptMeta : promptBody;

// Submit the prompt and print out the response
string response = await kernel.InvokePromptAsync<string>(
    prompt,
    new(new OpenAIPromptExecutionSettings() 
        { 
            MaxTokens = 400 
        })
    );
Console.WriteLine($"assistant >>> {response}");

assistant >>> {
  "Author": "Unknown",
  "Date": "Current Date and Time",
  "Title": "Understanding How to Specify JSON Schema",
  "Summary": "Explains the properties of a JSON schema for Article Summary in a concise manner for software engineers."
}
